In [7]:
import tensorflow as tf
import os

# Load the Keras model
model = tf.keras.models.load_model('S:/ML Course/Homework Week 9/bees-wasps.h5')

# Convert to TensorFlow Lite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Custom file path for saving the TFLite model
custom_path = 'S:/Model/bees-wasps.tflite' 

# Save the TFLite model to the custom path
with open(custom_path, 'wb') as f:
    f.write(tflite_model)

# Get the file size
file_size = os.path.getsize(custom_path)
print("TFLite Model File Size in Bytes:", file_size)


INFO:tensorflow:Assets written to: C:\Users\Detoxx\AppData\Local\Temp\tmp1py79bfr\assets


INFO:tensorflow:Assets written to: C:\Users\Detoxx\AppData\Local\Temp\tmp1py79bfr\assets


TFLite Model File Size in Bytes: 44866200


In [8]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='bees-wasps.tflite')
interpreter.allocate_tensors()

# Get output details
output_details = interpreter.get_output_details()
print("Output Index:", output_details[0]['index'])


Output Index: 13


In [3]:
from PIL import Image
from io import BytesIO
from urllib import request
import numpy as np

# Provided functions
def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

# Download and resize image
url = "https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg"
img = download_image(url)
target_size = (150, 150)  # Replace with the target size used in your model
img = prepare_image(img, target_size)

# Convert to numpy array and preprocess
img_array = np.array(img) / 255.0  # Assuming normalization was used in training
print("R channel value at the first pixel:", img_array[0, 0, 0])


R channel value at the first pixel: 0.9450980392156862


In [9]:
# Convert the image to required shape and type
input_data = np.expand_dims(img_array, axis=0).astype(np.float32)

# Load the TFLite model and run inference
interpreter = tf.lite.Interpreter(model_path='bees-wasps.tflite')
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]['index']
interpreter.set_tensor(input_index, input_data)
interpreter.invoke()

# Get the output
output_index = interpreter.get_output_details()[0]['index']
output_data = interpreter.get_tensor(output_index)
print("Model Output:", output_data[0][0])


Model Output: 0.65898407


In [14]:
import numpy as np
import tensorflow as tf
from PIL import Image
from io import BytesIO
from urllib import request

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size=(150, 150)):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    img_array = np.array(img) / 255.0
    return img_array

def load_model(model_path):
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

def predict(model, image_url):
    img = download_image(image_url)
    img_array = prepare_image(img)
    input_data = np.expand_dims(img_array, axis=0).astype(np.float32)

    input_index = model.get_input_details()[0]['index']
    model.set_tensor(input_index, input_data)
    model.invoke()

    output_index = model.get_output_details()[0]['index']
    output_data = model.get_tensor(output_index)
    return output_data[0][0]

# Load the model
model = load_model('C:/Users/Detoxx/bees-wasps.tflite')

# URL of the image to score
image_url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'

# Run the prediction
output = predict(model, image_url)
print("Model Output:", output)


Model Output: 0.65898407
